In [1]:
import numpy as np
import random
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix,
                             f1_score,
                             recall_score)
from sklearn.model_selection import train_test_split

In [2]:
# get working directory
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data')
model_dir = os.path.join(cwd, 'model')

# load data and pre-process datasets
train_df = pd.read_csv(os.path.join(data_dir, 'gptTestNames.csv'))
# test_df = pd.read_csv(os.path.join(data_dir, 'gptTestNames.csv'))
# val_df = pd.read_csv(os.path.join(data_dir, 'gptValNames.csv'))

In [3]:
# X_train = list()
# X_test = list()
# for race in ["API", "White", "Black", "Hispanic"]:
#     train, test  = train_test_split(train_df[train_df.label==race], 
#                                     train_size=300,
#                                     test_size=300, 
#                                     random_state=42)
#     X_train.append(train)
#     X_test.append(test)

X_train, X_test  = train_test_split(train_df, 
                                train_size=int(len(train_df) * 0.0008),
                                test_size=int(len(train_df) * 0.0002), 
                                random_state=42)

# X_train = pd.concat(X_train).sample(frac=1, random_state=10)
# X_test = pd.concat(X_test)

# eval_idx = [idx for idx in train_df.index if idx not in list(train.index) + list(test.index)]
# X_eval = train_df[train_df.index.isin(eval_idx)]
X_train, X_eval = train_test_split(X_train, 
                                test_size=0.1,
                                random_state=42)
X_train = X_train.reset_index(drop=True)

In [4]:
len(X_train), len(X_test), len(X_eval)

(1867, 518, 208)

In [5]:
def generate_prompt(data_point, shuffle=False):
    if not shuffle:
        return f"""
                Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Black, Hispanic, or White. 
                Your answer should only be the category name.
                [{data_point["name"]}].
                ANSWER: {data_point["label"]}
                """.strip()
    
    categories = ["Hispanic", "Black", "White", "Asian"]
    random.shuffle(categories)
    categories_str = ', '.join(categories)
    return f"""
            Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: {categories_str}. 
            Your answer should only be the category name.
            [{data_point["name"]}]
            ANSWER: {data_point["label"]}
            """.strip()

def generate_test_prompt(data_point, shuffle=False):
    if not shuffle:
        return f"""
                Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Black, Hispanic, or White. 
                Your answer should only be the category name.
                [{data_point["name"]}]
                ANSWER: """.strip()
    
    categories = ["Hispanic", "Black", "White", "Asian"]
    random.shuffle(categories)
    categories_str = ', '.join(categories)
    return f"""
            Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: {categories_str}. 
            Your answer should only be the category name.
            [{data_point["name"]}]
            ANSWER: """.strip()


X_train_1 = pd.DataFrame(X_train.apply(lambda row: generate_prompt(row, shuffle=True), axis=1), 
                       columns=["name"])
X_eval_1 = pd.DataFrame(X_eval.apply(lambda row: generate_prompt(row, shuffle=True), axis=1), 
                       columns=["name"])

y_true = X_test.label
X_test_1 = pd.DataFrame(X_test.apply(lambda row: generate_test_prompt(row, shuffle=True), axis=1), 
                      columns=["name"])
X_test_2 = pd.DataFrame(X_test.apply(lambda row: generate_test_prompt(row, shuffle=False), axis=1), 
                      columns=["name"])

train_data = Dataset.from_pandas(X_train_1)
eval_data = Dataset.from_pandas(X_eval_1)

In [6]:
def evaluate(y_true, y_pred):
    labels = ['API', 'Black', 'Hispanic', 'White']
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred, target_names=labels)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [7]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(test))):
    # for i in [69, 222, 676, 1270, 2060, 3684, 3827, 4472, 4799, 4972, 5120]:
        prompt = test.iloc[i]["name"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 4, 
                        # temperature = 0.01,
                        do_sample = False,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split(":")[-1].lower()
        # print(prompt, answer)
        if "asian" in answer:
            y_pred.append("API")
        elif "black" in answer:
            y_pred.append("Black")
        elif "hispanic" in answer:
            y_pred.append("Hispanic")
        elif "white" in answer:
            y_pred.append("White")
        else:
            y_pred.append("none")
            print(prompt,answer)
    return y_pred

In [10]:
y_pred = predict(X_test_1, model, tokenizer)
y_pred1 = predict(X_test_2, model, tokenizer)

  0%|          | 0/518 [00:00<?, ?it/s]/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 518/518 [07:10<00:00,  1.20it/s]


In [ ]:
unique_res = set(y_pred)  # Get unique labels
print(unique_res)

unique_labels = set(y_true)
print(unique_labels)

labels = list(set(list(set(y_true))+list(set(y_pred))))
labels

In [11]:
# y_pred = ['API' if 'Asia Pacific Islander' in x else x for x in y_pred]
evaluate(y_true, y_pred)
evaluate(y_true, y_pred1)

Accuracy: 0.463

Classification Report:
              precision    recall  f1-score   support

         API       0.07      0.55      0.13        11
       Black       0.20      0.46      0.28        80
    Hispanic       0.75      0.73      0.74        92
       White       0.80      0.39      0.52       335

    accuracy                           0.46       518
   macro avg       0.46      0.53      0.42       518
weighted avg       0.68      0.46      0.52       518


Confusion Matrix:
[[  6   4   1   0]
 [ 14  37   5  24]
 [  4  13  67   8]
 [ 56 133  16 130]]
Accuracy: 0.320

Classification Report:
              precision    recall  f1-score   support

         API       0.04      1.00      0.07        11
       Black       0.37      0.31      0.34        80
    Hispanic       0.93      0.72      0.81        92
       White       0.88      0.19      0.31       335

    accuracy                           0.32       518
   macro avg       0.55      0.56      0.38       518
weighted 

In [12]:
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score (Micro): {f1_micro:.3f}")
print(f"F1 Score (Macro): {f1_macro:.3f}")
print(f"F1 Score (Weighted): {f1_weighted:.3f}")

print("--------------below is for shuffle test----------------")
f1_micro = f1_score(y_true, y_pred1, average='micro')
f1_macro = f1_score(y_true, y_pred1, average='macro')
f1_weighted = f1_score(y_true, y_pred1, average='weighted')
print(f"F1 Score (Micro): {f1_micro:.3f}")
print(f"F1 Score (Macro): {f1_macro:.3f}")
print(f"F1 Score (Weighted): {f1_weighted:.3f}")

F1 Score (Micro): 0.463
F1 Score (Macro): 0.418
F1 Score (Weighted): 0.515
--------------below is for shuffle test----------------
F1 Score (Micro): 0.320
F1 Score (Macro): 0.383
F1 Score (Weighted): 0.401


In [13]:
print(X_test_1.iloc[0]["name"])
print(X_test_2.iloc[0]["name"])
print(X_train_1.iloc[0]["name"])

Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Hispanic, Asian, White, Black. 
            Your answer should only be the category name.
            [Saucedo Cesar]
            ANSWER:
Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Black, Hispanic, or White. 
                Your answer should only be the category name.
                [Saucedo Cesar]
                ANSWER:
Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: White, Asian, Hispanic, Black. 
            Your answer should only be the category name.
            [Tarnecki Lori]
            ANSWER: White


In [15]:
output_dir="trained_weigths"

peft_config = LoraConfig(
        lora_alpha=16, 
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=2,                       # number of training epochs
    per_device_train_batch_size=4,            # batch size per device during training
    gradient_accumulation_steps=1,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="name",
    tokenizer=tokenizer,
    max_seq_length=None,
    packing=False,
    dataset_batch_size=4,
    # dataset_kwargs={
    #     "add_special_tokens": False,
    #     "append_concat_token": False,
    # }
)

/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1867 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
trainer.train()

/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.354500,0.387827


TrainOutput(global_step=467, training_loss=0.504207287406513, metrics={'train_runtime': 1953.4371, 'train_samples_per_second': 0.956, 'train_steps_per_second': 0.239, 'total_flos': 4618404962795520.0, 'train_loss': 0.504207287406513, 'epoch': 1.0})

In [ ]:
trainer.model.save_pretrained(output_dir)
trainer.tokenizer.save_pretrained(output_dir)

In [17]:
model.eval()
y_pred = predict(X_test_1, model, tokenizer)
y_pred1 = predict(X_test_2, model, tokenizer)
# evaluate(y_true, y_pred)

  0%|          | 0/518 [00:00<?, ?it/s]/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  7%|▋         | 38/518 [00:29<06:13,  1.29it/s]

Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Hispanic, White, Black. 
            Your answer should only be the category name.
            [Vongrasamy Phouphet]
            ANSWER:  apia
           


  9%|▉         | 46/518 [00:35<06:14,  1.26it/s]

Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, White, Hispanic, Black. 
            Your answer should only be the category name.
            [Nguyen Nhuy]
            ANSWER:  apia
           


 67%|██████▋   | 348/518 [04:30<02:11,  1.29it/s]

...

 78%|███████▊  | 405/518 [05:14<01:27,  1.29it/s]

Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Black, Hispanic, or White. 
                Your answer should only be the category name.
                [Tufts Mikyong]
                ANSWER:  apia
           


 89%|████████▉ | 461/518 [05:58<00:44,  1.28it/s]

Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Black, Hispanic, or White. 
                Your answer should only be the category name.
                [Nagassar Sharmila]
                ANSWER:  api
            </


100%|██████████| 518/518 [06:42<00:00,  1.29it/s]


In [18]:
y_pred = ['API' if 'none' in x else x for x in y_pred]
y_pred1 = ['API' if 'none' in x else x for x in y_pred1]
evaluate(y_true, y_pred)
evaluate(y_true, y_pred1)


Accuracy: 0.811

Classification Report:
              precision    recall  f1-score   support

         API       1.00      0.45      0.62        11
       Black       0.67      0.17      0.28        80
    Hispanic       0.90      0.84      0.87        92
       White       0.80      0.97      0.87       335

    accuracy                           0.81       518
   macro avg       0.84      0.61      0.66       518
weighted avg       0.80      0.81      0.78       518


Confusion Matrix:
[[  5   0   1   5]
 [  0  14   4  62]
 [  0   0  77  15]
 [  0   7   4 324]]
Accuracy: 0.817

Classification Report:
              precision    recall  f1-score   support

         API       1.00      0.55      0.71        11
       Black       0.59      0.21      0.31        80
    Hispanic       0.90      0.86      0.88        92
       White       0.81      0.96      0.88       335

    accuracy                           0.82       518
   macro avg       0.82      0.64      0.69       518
weighted 

In [19]:
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score (Micro): {f1_micro:.3f}")
print(f"F1 Score (Macro): {f1_macro:.3f}")
print(f"F1 Score (Weighted): {f1_weighted:.3f}")
print("--------------below is for shuffle test----------------")
f1_micro = f1_score(y_true, y_pred1, average='micro')
f1_macro = f1_score(y_true, y_pred1, average='macro')
f1_weighted = f1_score(y_true, y_pred1, average='weighted')
print(f"F1 Score (Micro): {f1_micro:.3f}")
print(f"F1 Score (Macro): {f1_macro:.3f}")
print(f"F1 Score (Weighted): {f1_weighted:.3f}")

F1 Score (Micro): 0.811
F1 Score (Macro): 0.660
F1 Score (Weighted): 0.775
--------------below is for shuffle test----------------
F1 Score (Micro): 0.817
F1 Score (Macro): 0.694
F1 Score (Weighted): 0.788


In [ ]:
cm = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=['API','Black','Hispanic','White'])
metrics_per_class = {}
tpr, tnr = [], []
for i in range(len(cm)):
    TP = cm[i, i]
    FP = sum(cm[:, i]) - TP
    FN = sum(cm[i, :]) - TP
    TN = sum(cm.sum(axis=1)) - TP - FP - FN
    TPR = TP / float(TP + FN) if (TP + FN) > 0 else 0
    TNR = TN / float(TN + FP) if (TN + FP) > 0 else 0
    # class_label = le.inverse_transform([i])[0]  # Convert index back to original class label
    # metrics_per_class[class_label] = {'TPR': TPR, 'TNR': TNR}
    tpr.append(TPR)
    tnr.append(TNR)

temp = (np.array(tpr)+np.array(tnr))*0.5
# print(temp)
gap = 0.0
for i in range(len(temp)-1):
    gap += abs(temp[i]-temp[-1])
gap /= 3
print('1-GAP is:',1-gap)

In [ ]:
recalls = [0.09, 0.97, 0.4, 0.02]

dis = 0.0
for i in range(len(recalls)-1):
    dis += recalls[i]/recalls[-1]
dis /= (len(recalls)-1)
print('Disparate Impact is:',dis)